<a href="https://colab.research.google.com/github/Mahessh01/AI-Lab1-MaheshPharswan/blob/main/GoalBasedAgent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random
import numpy as np

class VacuumAgent:
    def __init__(self, grid_size=4, battery_capacity=100):
        self.grid_size = grid_size
        self.grid = np.random.choice([0, 1], size=(grid_size, grid_size))  # 0: clean, 1: dirty
        self.position = [0, 0]  # Start at top-left
        self.battery = battery_capacity
        self.actions = ['up', 'down', 'left', 'right', 'suck']
        self.total_dirt_cleaned = 0

    def is_valid_move(self, pos):
        return 0 <= pos[0] < self.grid_size and 0 <= pos[1] < self.grid_size

    def calculate_utility(self, action):
        x, y = self.position
        utility = 0

        # Battery cost for actions
        battery_cost = {'suck': 10, 'up': 5, 'down': 5, 'left': 5, 'right': 5}

        # If battery is too low, penalize actions except moving to recharge (simplified as staying put)
        if self.battery < battery_cost[action]:
            return -100

        if action == 'suck':
            if self.grid[x][y] == 1:
                utility += 50  # Reward for cleaning dirt
            else:
                utility -= 10  # Penalty for unnecessary suck
        else:
            # Calculate new position
            new_pos = [x, y]
            if action == 'up':
                new_pos[0] -= 1
            elif action == 'down':
                new_pos[0] += 1
            elif action == 'left':
                new_pos[1] -= 1
            elif action == 'right':
                new_pos[1] += 1

            if not self.is_valid_move(new_pos):
                utility -= 20  # Penalty for hitting wall
            else:
                # Check for nearby dirt
                for dx in [-1, 0, 1]:
                    for dy in [-1, 0, 1]:
                        nx, ny = new_pos[0] + dx, new_pos[1] + dy
                        if 0 <= nx < self.grid_size and 0 <= ny < self.grid_size:
                            if self.grid[nx][ny] == 1:
                                utility += 20 / (abs(dx) + abs(dy) + 1)  # Reward for moving closer to dirt

        # Battery penalty
        utility -= battery_cost[action]
        return utility

    def choose_action(self):
        utilities = {action: self.calculate_utility(action) for action in self.actions}
        best_action = max(utilities, key=utilities.get)
        return best_action

    def perform_action(self, action):
        x, y = self.position
        battery_cost = {'suck': 10, 'up': 5, 'down': 5, 'left': 5, 'right': 5}

        if self.battery < battery_cost[action]:
            print("Low battery! Action not performed.")
            return False

        if action == 'suck':
            if self.grid[x][y] == 1:
                self.grid[x][y] = 0
                self.total_dirt_cleaned += 1
                print(f"Cleaned dirt at {self.position}")
            self.battery -= battery_cost[action]
        else:
            new_pos = [x, y]
            if action == 'up':
                new_pos[0] -= 1
            elif action == 'down':
                new_pos[0] += 1
            elif action == 'left':
                new_pos[1] -= 1
            elif action == 'right':
                new_pos[1] += 1

            if self.is_valid_move(new_pos):
                self.position = new_pos
                self.battery -= battery_cost[action]
                print(f"Moved {action} to {self.position}")
            else:
                print("Hit wall! Staying in place.")
                self.battery -= battery_cost[action]  # Still consume battery for attempt

        return True

    def print_grid(self):
        for i in range(self.grid_size):
            row = ""
            for j in range(self.grid_size):
                if [i, j] == self.position:
                    row += " A "
                else:
                    row += " D " if self.grid[i][j] == 1 else " . "
            print(row)
        print(f"Battery: {self.battery}%")
        print(f"Dirt cleaned: {self.total_dirt_cleaned}\n")

def main():
    agent = VacuumAgent()
    steps = 20  # Limit steps for demonstration
    for _ in range(steps):
        agent.print_grid()
        if agent.battery <= 0:
            print("Battery depleted! Stopping.")
            break
        action = agent.choose_action()
        print(f"Chose action: {action}")
        agent.perform_action(action)
        if np.all(agent.grid == 0):
            print("All dirt cleaned!")
            break

if __name__ == "__main__":
    main()

 A  .  D  D 
 .  .  D  . 
 D  D  .  . 
 D  .  D  D 
Battery: 100%
Dirt cleaned: 0

Chose action: suck
Cleaned dirt at [0, 0]
 A  .  D  D 
 .  .  D  . 
 D  D  .  . 
 D  .  D  D 
Battery: 90%
Dirt cleaned: 1

Chose action: down
Moved down to [1, 0]
 .  .  D  D 
 A  .  D  . 
 D  D  .  . 
 D  .  D  D 
Battery: 85%
Dirt cleaned: 1

Chose action: down
Moved down to [2, 0]
 .  .  D  D 
 .  .  D  . 
 A  D  .  . 
 D  .  D  D 
Battery: 80%
Dirt cleaned: 1

Chose action: right
Moved right to [2, 1]
 .  .  D  D 
 .  .  D  . 
 D  A  .  . 
 D  .  D  D 
Battery: 75%
Dirt cleaned: 1

Chose action: suck
Cleaned dirt at [2, 1]
 .  .  D  D 
 .  .  D  . 
 D  A  .  . 
 D  .  D  D 
Battery: 65%
Dirt cleaned: 2

Chose action: left
Moved left to [2, 0]
 .  .  D  D 
 .  .  D  . 
 A  .  .  . 
 D  .  D  D 
Battery: 60%
Dirt cleaned: 2

Chose action: suck
Cleaned dirt at [2, 0]
 .  .  D  D 
 .  .  D  . 
 A  .  .  . 
 D  .  D  D 
Battery: 50%
Dirt cleaned: 3

Chose action: down
Moved down to [3, 0]
 .  .  D  D 
 .